In [12]:
#Fine-Tuning BERT with a Logistic Regression Layer
import torch
from torch import nn
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [13]:
dataset = load_dataset("imdb")

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [15]:
import torch
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Load IMDB dataset
dataset = load_dataset("imdb")

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Balance the dataset to include both classes
def balance_classes(dataset, num_samples):
    class_0 = [sample for sample in dataset if sample["label"] == 0][:num_samples // 2]
    class_1 = [sample for sample in dataset if sample["label"] == 1][:num_samples // 2]
    return class_0 + class_1

balanced_train_data = balance_classes(tokenized_datasets["train"], 100)
balanced_eval_data = balance_classes(tokenized_datasets["test"], 50)

# Define the training arguments with optimizations
training_args = TrainingArguments(
    output_dir="./results",                 # Output directory
    evaluation_strategy="no",               # Disable evaluation during training
    learning_rate=2e-5,                     # Learning rate
    per_device_train_batch_size=4,          # Smaller batch size for faster training
    per_device_eval_batch_size=4,           # Smaller batch size for evaluation
    num_train_epochs=1,                     # Reduce number of epochs to 1
    fp16=True,                              # Enable mixed precision training
    max_steps=100,                          # Limit steps for faster execution
)

# Load the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Function to extract embeddings (logits) from BERT
def extract_embeddings(model, dataset):
    embeddings = []
    labels = []
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for sample in dataset:  # Loop through each item in the dataset
            input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0)  # Add batch dimension
            attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0)  # Add batch dimension
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.cpu().numpy()
            embeddings.append(logits.flatten())  # Append flattened logits
            labels.append(sample["label"])  # Append label
    return np.array(embeddings), np.array(labels)

# Extract embeddings from the train and test datasets
train_embeddings, train_labels = extract_embeddings(model, balanced_train_data)
test_embeddings, test_labels = extract_embeddings(model, balanced_eval_data)

# Train a logistic regression model on top of the embeddings
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(train_embeddings, train_labels)

# Predict and evaluate
train_preds = log_reg.predict(train_embeddings)
test_preds = log_reg.predict(test_embeddings)

train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")



/home/j/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Accuracy: 0.61
Test Accuracy: 0.78


In [16]:
# Initialize the Trainer for BERT
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=balanced_train_data,
    eval_dataset=balanced_eval_data,
)

# Train the model (BERT fine-tuning)
trainer.train()


# Save the trained BERT model and tokenizer for future use
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")


max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json')

In [17]:
# Function to evaluate model accuracy
def evaluate_model(model, dataset):
    model.eval()  # Set model to evaluation mode
    predictions, labels = [], []
    with torch.no_grad():
        for sample in dataset:
            input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0)  # Add batch dimension
            attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0)  # Add batch dimension
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=-1).cpu().numpy()
            
            predictions.extend(pred)  # Append the prediction directly
            labels.append(sample["label"])  # Append the label directly (no need to iterate)
    
    accuracy = accuracy_score(labels, predictions)
    return accuracy

# Evaluate on the test set
initial_accuracy = evaluate_model(model, balanced_eval_data)
print(f"Accuracy before Logistic Regression: {initial_accuracy}")


Accuracy before Logistic Regression: 0.8


In [18]:

# Evaluate the fine-tuned model on the test set
trainer.evaluate()

# Predict using the fine-tuned BERT model
def predict(texts, model, tokenizer):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    return predictions.numpy()

# Example prediction
texts = ["I love this movie!", "This movie was terrible."]
predictions = predict(texts, model, tokenizer)
print("Predictions:", predictions)  # Output will be the predicted labels (0 or 1)


Predictions: [0 0]


In [19]:
trainer

In [20]:
predictions

array([0, 0])

In [21]:
texts

['I love this movie!', 'This movie was terrible.']

In [22]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [23]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [24]:
# Predict using the fine-tuned BERT model
def predict(texts, model, tokenizer):
    # Encode the input texts using the tokenizer
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
    
    # Ensure the model is in evaluation mode
    model.eval()

    with torch.no_grad():
        # Forward pass through the model to get logits
        outputs = model(**encodings)
        logits = outputs.logits
        
        # Convert logits to probabilities using softmax (optional but useful for multi-class)
        probs = torch.nn.functional.softmax(logits, dim=-1)
        
        # Get the predicted class (index of the highest probability)
        predictions = torch.argmax(probs, dim=-1)
    
    return predictions.numpy()

# Example prediction
texts = ["I love this movie!", "This movie was terrible."]
predictions = predict(texts, model, tokenizer)
print("Predictions:", predictions)  # Output should show the predicted labels (0 or 1)


Predictions: [0 0]


In [25]:
# Example prediction
texts = [
    "I love this movie! It's amazing and so entertaining.",
    "This movie was terrible. I hated every minute of it.",
    "The plot was great, but the acting could have been better.",
    "A fantastic film with breathtaking visuals and a captivating story!",
    "I was really bored throughout this film. It's not worth watching.",
    "One of the best movies I’ve seen this year. I highly recommend it!",
    "The movie started off strong but ended poorly.",
    "Not bad, but not great either. Just an average movie.",
    "I couldn't stop laughing at the jokes. This was a really funny film.",
    "It was too predictable and lacked depth. Would not watch again.",
    "What a disappointment! The trailer was way better than the actual movie.",
    "I’m glad I watched it, but I wouldn’t watch it again.",
    "Incredible! A masterpiece that will be remembered for years.",
    "This movie is a must-see for any fan of action films.",
    "Absolutely awful. I don’t understand the hype around it.",
    "The soundtrack was amazing, and the acting was top-notch.",
    "This was a complete waste of time. I would not recommend it to anyone.",
    "The movie was too long and dragged on. Could have been much shorter.",
    "A rollercoaster of emotions. I loved every second of it.",
    "It’s a fun movie to watch with friends, but not the best out there.",
    "A very emotional and touching story that left me in tears.",
    "The special effects were spectacular, but the story was lacking.",
    "I didn’t connect with the characters at all, but the film was well made.",
    "If you're looking for something lighthearted, this movie is perfect.",
    "A complete masterpiece from start to finish. I loved everything about it!",
    "This movie is definitely overrated. I don’t see what the fuss is about."
]

# Predictions
predictions = predict(texts, model, tokenizer)
print("Predictions:", predictions)  # Output will be the predicted labels (0 or 1)


Predictions: [1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0]


In [49]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
from random import shuffle
import numpy as np
import time  # Import time module to track duration

# 1. Load the full dataset without reducing the size
dataset = load_dataset("imdb", split='train[:10%]')  # Use a small portion (10%) to avoid too large data
test_dataset = load_dataset("imdb", split='test[:10%]')  # Use 10% of the test data

# 2. Load the tokenizer and model (DistilBERT)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 3. Tokenize the dataset with a smaller max_length
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncatValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0ion=True, max_length=32)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_test_data = test_dataset.map(tokenize_function, batched=True)

# 4. Custom balance function (if necessary) - random oversampling or undersampling
def balance_classes(dataset, num_samples):
    # Separate by class labels
    class_0 = [sample for sample in dataset if sample["label"] == 0]
    class_1 = [sample for sample in dataset if sample["label"] == 1]
    
    # If one class is missing, handle it by adding a small sample from the available class
    if len(class_0) == 0 or len(class_1) == 0:
        print("One class is missing. Using available class only.")
        return dataset  # Return the dataset as is if one class is missing
    
    # Ensure balanced classes by resampling the smaller class
    if len(class_0) < len(class_1):
        class_0 = class_0 * (len(class_1) // len(class_0)) + class_0[:len(class_1) % len(class_0)]
    elif len(class_1) < len(class_0):
        class_1 = class_1 * (len(class_0) // len(class_1)) + class_1[:len(class_0) % len(class_1)]
    
    balanced_data = class_0 + class_1  # Combine both classes
    shuffle(balanced_data)  # Shuffle the dataset to ensure randomness
    
    # Check class distribution after balancing
    class_0_count = sum([1 for sample in balanced_data if sample["label"] == 0])
    class_1_count = sum([1 for sample in balanced_data if sample["label"] == 1])
    print(f"Class 0 samples: {class_0_count}, Class 1 samples: {class_1_count}")
    
    return balanced_data

# Apply custom balancing
balanced_train_data = balance_classes(tokenized_datasets, 1000)
balanced_test_data = balance_classes(tokenized_test_data, 500)

# 5. Extract embeddings (features) from DistilBERT with time tracking
def extract_embeddings(model, dataset):
    embeddings = []
    labels = []
    model.eval()  # Set the model to evaluation mode
    
    start_time = time.time()  # Record the start time for the entire extraction process
    
    with torch.no_grad():
        for i, sample in enumerate(dataset):
            epoch_start_time = time.time()  # Record the time at the start of each step
            
            input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0)  # Add batch dimension
            attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0)  # Add batch dimension
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.cpu().numpy()
            embeddings.append(logits.flatten())  # Append flattened logits
            labels.append(sample["label"])  # Append label
            
            epoch_end_time = time.time()  # Record the time after each step
    
    total_end_time = time.time()  # Record the total time taken for all steps
    print(f"Total time for extracting embeddings: {total_end_time - start_time:.4f} seconds.")
    
    return np.array(embeddings), np.array(labels)

# Extract embeddings from the balanced train and test datasets
train_embeddings, train_labels = extract_embeddings(model, balanced_train_data)
test_embeddings, test_labels = extract_embeddings(model, balanced_test_data)

# 6. Display the class distribution
unique_train_labels = np.unique(train_labels)
print(f"Classes in the training data: {unique_train_labels}")

# 7. Final output with reduced data
print(f"Train embeddings shape: {train_embeddings.shape}")
print(f"Test embeddings shape: {test_embeddings.shape}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


One class is missing. Using available class only.
One class is missing. Using available class only.
Total time for extracting embeddings: 144.7922 seconds.
Total time for extracting embeddings: 157.4270 seconds.
Classes in the training data: [0]
Train embeddings shape: (2500, 2)
Test embeddings shape: (2500, 2)


In [50]:
# Function to evaluate model accuracy
def evaluate_model(model, dataset):
    model.eval()  # Set model to evaluation mode
    predictions, labels = [], []
    with torch.no_grad():
        for sample in dataset:
            input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0)  # Add batch dimension
            attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0)  # Add batch dimension
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=-1).cpu().numpy()
            
            predictions.extend(pred)  # Append the prediction directly
            labels.append(sample["label"])  # Append the label directly (no need to iterate)
    
    accuracy = accuracy_score(labels, predictions)
    return accuracy

# Evaluate on the test set
initial_accuracy = evaluate_model(model, balanced_eval_data)
print(f"Accuracy before Logistic Regression: {initial_accuracy}")

Accuracy before Logistic Regression: 0.5
